In [1]:
from __future__ import division
import os
import sys

from collections import defaultdict

import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline

import gffutils as gfu
import pysam
from sqlite3 import OperationalError

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## 1) Get some general info on the yeast transcriptome. 

#### 1b) Create a database using gffutils (you can install gffutils via pip). Remember to only make a database once!

In [2]:
# %%time
try:
    db = gfu.create_db('saccharomyces_cerevisiae.gff', dbfn='sacCer.db')
except OperationalError:
    pass

#### 1c) Use gffutils to figure out all the types of feature that are in the gff

In [3]:
## create_db will return FeatureDB object, but incase this is not the first time, load db
db = gfu.FeatureDB(dbfn='sacCer.db')

In [4]:
featureTypes = [x for x in db.featuretypes()]
print ', '.join(featureTypes)

ARS, ARS_consensus_sequence, CDS, LTR_retrotransposon, W_region, X_element, X_element_combinatorial_repeat, X_region, Y_prime_element, Y_region, Z1_region, Z2_region, blocked_reading_frame, centromere, centromere_DNA_Element_I, centromere_DNA_Element_II, centromere_DNA_Element_III, chromosome, external_transcribed_spacer_region, five_prime_UTR_intron, gene, intein_encoding_region, internal_transcribed_spacer_region, intron, long_terminal_repeat, mRNA, mating_type_region, matrix_attachment_site, ncRNA_gene, non_transcribed_region, noncoding_exon, origin_of_replication, plus_1_translational_frameshift, pseudogene, rRNA_gene, region, silent_mating_type_cassette_array, snRNA_gene, snoRNA_gene, tRNA_gene, telomerase_RNA_gene, telomere, telomeric_repeat, transposable_element_gene


#### 1d) Use gffutils to figure out which which genes have introns. What fraction of genes have introns? Print a list of gene names for genes that have introns. Protip: note that there is an intron type, and that introns have mRNAs as parents.

In [5]:
allGenes = []
genesWithIntrons = []
for parentChildren in db.iter_by_parent_childs(featuretype='mRNA'):
    geneName = parentChildren[0].attributes['Name'][0].split('_')[0]
    allGenes.append(geneName)
    for child in parentChildren[1:]:
        if child.featuretype == 'intron':
            genesWithIntrons.append(geneName)

print '{} / {} genes have introns.'.format(len(genesWithIntrons), len(allGenes))
print
print ', '.join(genesWithIntrons)

314 / 6600 genes have introns.

YAL030W, YAL003W, YAL001C, YBL111C, YBL091C-A, YBL087C, YBL059C-A, YBL059W, YBL050W, YBL040C, YBL027W, YBL026W, YBL018C, YBR048W, YBR062C, YBR078W, YBR082C, YBR084C-A, YBR090C, YBR111W-A, YBR111W-A, YBR119W, YBR181C, YBR186W, YBR189W, YBR191W, YBR215W, YBR219C, YBR230C, YBR255C-A, YCL012C, YCL005W-A, YCL005W-A, YCL002C, YCR028C-A, YCR031C, YCR097W, YCR097W, YDL219W, YDL191W, YDL136W, YDL130W, YDL125C, YDL115C, YDL108W, YDL083C, YDL082W, YDL079C, YDL075W, YDL064W, YDL029W, YDL012C, YDR005C, YDR025W, YDR059C, YDR064W, YDR092W, YDR129C, YDR139C, YDR305C, YDR318W, YDR367W, YDR381W, YDR381C-A, YDR397C, YDR424C, YDR424C, YDR447C, YDR450W, YDR471W, YDR500C, YDR535C, YEL076C-A, YEL012W, YEL003W, YER003C, YER007C-A, YER014C-A, YER044C-A, YER056C-A, YER074W, YER074W-A, YER074W-A, YER093C-A, YER117W, YER133W, YER179W, YFL039C, YFL034C-B, YFL034C-A, YFL031W, YFR024C-A, YFR031C-A, YFR045W, YGL251C, YGL232W, YGL226C-A, YGL183C, YGL178W, YGL137W, YGL103W, YGL087C, YGL0

#### 1e) Compute the length of every gene in the yeast genome, and output a file where the first column is gene name, and the second column is length. Warning: you have to be careful for the genes with introns! You only want the sum of the CDS lengths, and you don’t want to count the intron lengths.

In [51]:
geneLengths = defaultdict(int)
for parentChildren in db.iter_by_parent_childs(featuretype='mRNA'):
    geneName = parentChildren[0].attributes['Name'][0].split('_')[0]
    for child in parentChildren[1:]:
        if child.featuretype == 'CDS':
            geneLengths[geneName] += (child.end - child.start)+1

with open('sacCer_geneLength.txt', 'w') as fh:
    for (g,l) in geneLengths.items():
        fh.write('{}\t{}\n'.format(g,l))

## 2) Map a yeast RNAseq experiment.

## 3) Quantify expression using gffutils and pysam

#### 3b) Quantify expression for each gene using pysam. You could use the pileup() method of an AlignmentFile, like you did when building the SNP caller. Alternatively, you could try to figure out the .count_coverage() method of an AlignmentFile which could make it easier. There is documentation of count_coverage() on the readthedocs.

In [7]:
srrBam = pysam.AlignmentFile('SRR/SRR1177156.sorted.bam')

In [8]:
geneExpression = {}
for gene in db.features_of_type('mRNA'):
    geneName = gene.attributes['Name'][0].split('_')[0]
#     geneExpression[geneName] = srrBam.count_coverage(reference=gene.seqid, start=gene.start, end=gene.end)
    try:
        pileup = srrBam.pileup(reference=gene.seqid, start=gene.start, end=gene.end)
    except ValueError:
        continue
    geneExpression[geneName] = len([x for x in pileup])

In [9]:
## Preview
geneExpression.items()[:10]

[('YAL008W', 0),
 ('YBR255W', 0),
 ('YGR164W', 50),
 ('YGR131W', 0),
 ('YNL003C', 0),
 ('YBR135W', 46),
 ('YBR160W', 0),
 ('YJL082W', 97),
 ('YJL142C', 0),
 ('YPL191C', 44)]

#### 3c) Output the FPKM for each gene into a file where one column is a gene name, and the other column is the FPKM. Note that outputting FPKM means that you need to divide the counts of each gene by its length, and normalize by the total number of reads that map to genes (so that means you should keep a counter of how many reads mapped to any genes as you iterate over gene in step b). 

In [57]:
totalReadsMappedinMillions = sum(geneExpression.values()) / 1e6
srrFPKM = {}
for gene,count in geneExpression.items():
    srrFPKM[gene] = (count / ((geneLengths[gene]/1000) * totalReadsMappedinMillions)) * 1e9

## 4) Find differential expression expression between two yeast strains.

#### 4c) Using gffutils and pysam, test for differential expression of every gene in the genome

##### RM strain

In [11]:
rmBam = pysam.AlignmentFile('RM/RM.sorted.bam')

In [12]:
rmExpression = {}
for gene in db.features_of_type('mRNA'):
    geneName = gene.attributes['Name'][0].split('_')[0]
    try:
        pileup = rmBam.pileup(reference=gene.seqid, start=gene.start, end=gene.end)
    except ValueError:
        continue
    rmExpression[geneName] = len([x for x in pileup])

In [14]:
rmDataFrame = pd.DataFrame(rmExpression.items(), columns=['gene_name','rm_count'])

In [15]:
rmDataFrame.head()

,gene_name,rm_count
0,YAL008W,100
1,YBR255W,50
2,YGR164W,68
3,YGR131W,99
4,YNL003C,190


##### BY strain

In [16]:
byBam = pysam.AlignmentFile('BY/BY.sorted.bam')

In [17]:
byExpression = {}
for gene in db.features_of_type('mRNA'):
    geneName = gene.attributes['Name'][0].split('_')[0]
    try:
        pileup = byBam.pileup(reference=gene.seqid, start=gene.start, end=gene.end)
    except ValueError:
        continue
    byExpression[geneName] = len([x for x in pileup])

In [18]:
byDataFrame = pd.DataFrame(byExpression.items(), columns=['gene_name','by_count'])

In [19]:
byDataFrame.head()

,gene_name,by_count
0,YAL008W,49
1,YBR255W,0
2,YGR164W,50
3,YGR131W,0
4,YNL003C,50


#### Merge with gene lengths

In [53]:
geneLengthsDataFrame = pd.DataFrame(geneLengths.items(), columns=['gene_name','gene_length'])

In [54]:
combined = geneLengthsDataFrame.merge(rmDataFrame, on='gene_name', how='outer').merge(byDataFrame, on='gene_name', how='outer')

In [55]:
combined.head()

,gene_name,gene_length,rm_count,by_count
0,YAL008W,597,100.0,49.0
1,YBR255W,2085,50.0,0.0
2,YGR164W,336,68.0,50.0
3,YGR131W,525,99.0,0.0
4,YNL003C,855,190.0,50.0


In [56]:
combined[pd.isnull(combined['rm_count']) | pd.isnull(combined['by_count'])] ## These must be MT genes, which pileup had issues with

,gene_name,gene_length,rm_count,by_count
204,Q0092,141,NaN,NaN
688,Q0075,1065,NaN,NaN
771,Q0297,156,NaN,NaN
1019,Q0130,231,NaN,NaN
1856,Q0032,291,NaN,NaN
2086,Q0010,387,NaN,NaN
2087,Q0017,162,NaN,NaN
2092,Q0275,810,NaN,NaN
2301,Q0070,1893,NaN,NaN
2536,Q0055,2565,NaN,NaN


In [28]:
combined.to_csv('../pset7/yeast_results.txt', sep='\t', index=False)

#### Calculate FPKM

In [29]:
filtered = combined[pd.notnull(combined['rm_count']) & pd.notnull(combined['by_count'])]

In [32]:
filtered.head()

,gene_name,gene_length,rm_count,by_count
0,YAL008W,597,100.0,49.0
1,YBR255W,2085,50.0,0.0
2,YGR164W,336,68.0,50.0
3,YGR131W,525,99.0,0.0
4,YNL003C,855,190.0,50.0


In [ ]:
totalReadsMappedinMillions = sum(geneExpression.values()) / 1e6
srrFPKM = {}
for gene,count in geneExpression.items():
    srrFPKM[gene] = (count / ((geneLengths[gene]/1000) * totalReadsMappedinMillions)) * 1e9

In [67]:
filtered['rm_fpkm'] = (filtered['rm_count'] / ((filtered['gene_length']/1000) * (filtered['rm_count'].sum() / 1e6))) * 1e9
filtered['by_fpkm'] = (filtered['by_count'] / ((filtered['gene_length']/1000) * (filtered['by_count'].sum() / 1e6))) * 1e9

In [68]:
filtered.head()

,gene_name,gene_length,rm_count,by_count,rm_fpkm,by_fpkm
0,YAL008W,597,100.0,49.0,1.229970e+11,7.671821e+10
1,YBR255W,2085,50.0,0.0,1.760892e+10,0.000000e+00
2,YGR164W,336,68.0,50.0,1.486067e+11,1.390937e+11
3,YGR131W,525,99.0,0.0,1.384665e+11,0.000000e+00
4,YNL003C,855,190.0,50.0,1.631760e+11,5.466138e+10
